# Handover Tracjectory Player 

In [1]:
import sys
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
sys.path.append('../package/helper/')
sys.path.append('../package/mujoco_usage/')

# Mujoco Related
from mujoco_parser import MuJoCoParserClass
from transformation import *
from mujoco_parser import solve_ik

## Initiate Mujoco

In [2]:
xml_path = '../data/assets/ur5e/handover_scene.xml'
env = MuJoCoParserClass(name='handover',rel_xml_path=xml_path,verbose=False)

joint_names_ur5e = ['shoulder_pan_joint','shoulder_lift_joint','elbow_joint','wrist_1_joint','wrist_2_joint','wrist_3_joint']
idxs_fwd_ur5e   = env.get_idxs_fwd(joint_names=joint_names_ur5e)

# Initail pose
init_joint_value = [-1.49194225, -1.42738526,  2.13327598, -0.70220845, -0.0491743,  -0.00317383]
env.reset()
env.forward(q=init_joint_value, joint_idxs=idxs_fwd_ur5e)
print ("Ready.")

Ready.


## Load Human Handover Data

In [3]:
# moderate
pose_data_dir = "../data/whole_motions/"

pose_data_names = os.listdir(pose_data_dir)
# sort names by number
pose_data_names = sorted(pose_data_names, key=lambda x: int(x.split("_")[2].split(".")[0]))
print(pose_data_names) # ID motions: motion_normal_xxx.pkl / OOD motions: motion_variation_xxx.pkl

# get paths
pose_data_paths = []
for pose_data_name in pose_data_names:
    pose_data_paths.append([pose_data_dir + "/" + pose_data_name])
pose_data_paths = sum(pose_data_paths, [])# sqeeze

# open pickle files and load data
motion_data_list = []
for pose_data_path in pose_data_paths:
    with open(pose_data_path, 'rb') as f:
        pose_data = pickle.load(f)
    motion_data_list.append(pose_data)
len(motion_data_list)


['motion_normal_0.pkl', 'motion_variation_0.pkl', 'motion_normal_1.pkl', 'motion_variation_1.pkl', 'motion_normal_2.pkl', 'motion_variation_2.pkl', 'motion_normal_3.pkl', 'motion_variation_3.pkl', 'motion_variation_4.pkl', 'motion_normal_4.pkl', 'motion_variation_5.pkl', 'motion_normal_5.pkl', 'motion_variation_6.pkl', 'motion_normal_6.pkl', 'motion_variation_7.pkl', 'motion_normal_7.pkl', 'motion_normal_8.pkl', 'motion_variation_8.pkl', 'motion_normal_9.pkl', 'motion_variation_9.pkl', 'motion_normal_10.pkl', 'motion_variation_10.pkl', 'motion_normal_11.pkl', 'motion_variation_11.pkl', 'motion_normal_12.pkl', 'motion_variation_12.pkl', 'motion_normal_13.pkl', 'motion_variation_13.pkl', 'motion_normal_14.pkl', 'motion_variation_14.pkl', 'motion_normal_15.pkl', 'motion_variation_15.pkl', 'motion_normal_16.pkl', 'motion_variation_16.pkl', 'motion_normal_17.pkl', 'motion_variation_17.pkl', 'motion_variation_18.pkl', 'motion_normal_18.pkl', 'motion_variation_19.pkl', 'motion_normal_19.pkl',

1000

## ZED Bodytracking (34) Kinematic Sturcture

In [4]:
leg_joint_list = [18, 19, 20, 21, 32, # right leg
           22, 23, 24, 25, 33 # left leg
]

ks_body_receiver = [[0,1], [1,2], [2,3], [3,26], [26,27], [27,28], [28,29], [27,30], [27,31], # middle
           [2,4], [4,5], [5,6], [6,7], [7,8], [8,9], [7,10], # right arm
           [2,11], [11,12], [12,13], [13,14], [14,15], [15,16], [14,17], # left arm,
           [0,18], [18,19], [19,20], [20,21], [20,32], # right leg
           [0,22], [22,23], [23,24], [24,25], [24,33], # left leg
]
joint_idx_list_receiver = range(34)

ks_body_giver = [[0,1], [1,2], [2,3], [3,26], [26,27], [27,28], [28,29], [27,30], [27,31], # middle
           [2,4], [4,5], [5,6], [6,7], [7,8], [8,9], [7,10], # right arm
           [2,11], [11,12], [12,13], [13,14], [14,15], [15,16], [14,17], # left arm,
]
joint_idx_list_giver = [i for i in range(34) if i not in leg_joint_list]

# allocate rgba data for each idxs
rgba = {}
rgba["receiver"] = (1, 0, 0, 1)
rgba["giver"] = (0, 0, 1, 1)
rgba["object"] = (0, 0, 1, 0.3)

## Extract Motion Data

In [5]:
def motion_info_extraction(motion_data_list, motion_idx):
    motion_info = motion_data_list[motion_idx]
    pose_giver = np.array(motion_info["pose_giver"])            # (motion_len, 34, 3)
    pose_receiver = np.array(motion_info["pose_receiver"])      # (motion_len, 34, 3)
    pose_object = np.array(motion_info["pose_object"])          # (motion_len, 3)
    quat_object = np.array(motion_info["quat_object"])          # (motion_len, 4)
    len_motion = len(pose_object)

    return pose_giver, pose_receiver, pose_object, quat_object, len_motion

motion_idx = random.randint(0, len(motion_data_list)-1)
pose_giver, pose_receiver, pose_object, quat_object, len_motion = motion_info_extraction(motion_data_list, motion_idx)

print(pose_giver.shape)
print(pose_receiver.shape)
print(pose_object.shape)
print(quat_object.shape)

(137, 34, 3)
(137, 34, 3)
(137, 3)
(137, 4)


## R2H Motion Visualizer

In [6]:
env.reset()
env.forward()
env.init_viewer()
env.set_viewer(azimuth=90,distance=1.5,elevation=-10,lookat=[.8, 0., 1.2])

render_dt = 1./30.
motion_dt = 1./30.

p_text = [0,0,2]
t_s = time.perf_counter()
t_m = t_s
pose_idx = 0
p_pose_idx = np.array([1.0, 0., 1.2])

while env.is_viewer_alive():
    t_now = time.perf_counter()
    if t_now - t_m >= motion_dt:
        t_m += motion_dt
        if pose_idx == len_motion-1:
            pose_idx = 0
            motion_idx = random.randint(0, len(motion_data_list)-1)
            pose_giver, pose_receiver, pose_object, quat_object, len_motion = motion_info_extraction(motion_data_list, motion_idx)
        else: 
            pose_idx += 1
        p_object = pose_object[pose_idx]
        R_object = quat2r(quat_object[pose_idx])
    if t_now - t_s >= render_dt:
        t_s += render_dt

        # Forward Robot Motion
        orientation_vector = p_object - env.get_p_body("ur_base")
        R_trgt = rpy2r([0, 0,np.arctan2(orientation_vector[1], orientation_vector[0])-np.pi/2]) @ rpy2r([0,np.pi,0])
        p_trgt = p_object - np.array([0,0,0.1])
        # Clip p_trgt for stable control # max_extension = 0.828, min_extension = 0.47
        extension = np.linalg.norm(p_trgt - env.get_p_body("ur_shoulder_link"))
        if extension > 0.828:
            p_trgt = env.get_p_body("ur_shoulder_link") + (p_trgt - env.get_p_body("ur_shoulder_link")) / extension * 0.828
        elif extension < 0.5:
            p_trgt = env.get_p_body("ur_shoulder_link") + (p_trgt - env.get_p_body("ur_shoulder_link")) / extension * 0.5
        # solve ik
        if pose_idx == 0:
            q_init = init_joint_value
        else:
            q_init = env.get_qpos_joints(joint_names_ur5e)
        q,_,_ = solve_ik(
                            env                = env,
                            joint_names_for_ik = joint_names_ur5e,
                            body_name_trgt     = 'ur_tcp_link',
                            q_init             = q_init,
                            p_trgt             = p_trgt,
                            R_trgt             = R_trgt,
                        )
        # forward and plot target pose
        env.forward(q=q, joint_idxs=idxs_fwd_ur5e)
        env.plot_T(p=p_trgt, R=R_trgt, axis_len=0.1, label= "target_pose")

        # Plot Time and Index
        env.plot_text(p=pose_giver[pose_idx][27] + np.array([0,0,0.1]),label=f"motion index: {motion_idx}, time: {(pose_idx*motion_dt):.2f}s")

        # Plot Object
        p_object_center = p_object - R_object @ np.array([0,0,0.1])
        env.plot_cylinder(p=p_object_center, R=R_object, r=0.04, h=0.10, rgba=rgba["object"])

        # Plot Receiver Body
        for ks in ks_body_receiver:
            env.plot_cylinder_fr2to(p_fr=pose_receiver[pose_idx][ks[0]], p_to=pose_receiver[pose_idx][ks[1]], r=0.005, rgba=rgba["receiver"])
        for i in joint_idx_list_receiver:
            env.plot_sphere(p=pose_receiver[pose_idx][i], r=0.02, rgba=rgba["receiver"])
        # Plot Receiver Traj (base)
        env.plot_traj(pose_receiver[::10,1,:], rgba=rgba["receiver"], plot_cylinder=True, cylinder_r=0.003)

        # Plot Giver Body
        # for ks in ks_body_giver:
        #     env.plot_cylinder_fr2to(p_fr=pose_giver[pose_idx][ks[0]], p_to=pose_giver[pose_idx][ks[1]], r=0.005, rgba=rgba["giver"])
        # for i in joint_idx_list_giver:
        #     env.plot_sphere(p=pose_giver[pose_idx][i], r=0.02, rgba=rgba["giver"])
        # Plot Object Traj (base)
        env.plot_traj(pose_object[::10], rgba=rgba["object"], plot_cylinder=True, cylinder_r=0.003)
        
        env.render()

ik_err:[0.8432] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[100]
